# Neural Networks with PyTorch

In this assignment, we are going to train a Neural Networks on the Japanese MNIST dataset. It is composed of 70000 images of handwritten Hiragana characters. The target variables has 10 different classes.

Each image is of dimension 28 by 28. But we will flatten them to form a dataset composed of vectors of dimension (784, 1). The training process will be similar as for a structured dataset.

# 1. Import Required Packages

In [ ]:
from google.colab import drive
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# 2. Dataset from Google Drive

Dataset extracted from personal Google Drive.


[2.1] Mount Google Drive

In [ ]:
drive.mount('/content/gdrive')

In [ ]:
! mkdir -p /content/gdrive/MyDrive/DL_ASG_1

In [ ]:
%cd '/content/gdrive/MyDrive/DL_ASG_1'

In [ ]:
!ls

[2.4] Dowload the dataset files to your Google Drive if required

In [ ]:
import requests
from tqdm import tqdm
import os.path

def download_file(url):
    path = url.split('/')[-1]
    if os.path.isfile(path):
        print (f"{path} already exists")
    else:
      r = requests.get(url, stream=True)
      with open(path, 'wb') as f:
          total_length = int(r.headers.get('content-length'))
          print('Downloading {} - {:.1f} MB'.format(path, (total_length / 1024000)))
          for chunk in tqdm(r.iter_content(chunk_size=1024), total=int(total_length / 1024) + 1, unit="KB"):
              if chunk:
                  f.write(chunk)

url_list = [
    'http://codh.rois.ac.jp/kmnist/dataset/kmnist/kmnist-train-imgs.npz',
    'http://codh.rois.ac.jp/kmnist/dataset/kmnist/kmnist-train-labels.npz',
    'http://codh.rois.ac.jp/kmnist/dataset/kmnist/kmnist-test-imgs.npz',
    'http://codh.rois.ac.jp/kmnist/dataset/kmnist/kmnist-test-labels.npz'
]

for url in url_list:
    download_file(url)

[2.5] List the content of the folder and confirm files have been dowloaded properly

In [ ]:
! ls

# 3. Load Data

[3.1] Importing the required modules from PyTorch

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import TensorDataset, DataLoader

In [ ]:
torch.__version__

[3.2]Creating 2 variables called `img_height` and `img_width` that will both take the value 28

In [ ]:
img_height = 28
img_width = 28

[3.3] Create a function that loads a .npz file using numpy and return the content of the `arr_0` key

In [ ]:
def load(f):
    return np.load(f)['arr_0']

This function provides a simple way to load a .npz file and directly access the first array stored in it.


[3.4] Load the 4 files saved on your Google Drive into their respective variables: x_train, y_train, x_test and y_test

In [ ]:
x_train = load('/content/gdrive/MyDrive/DL_ASG_1/kmnist-train-imgs.npz')
y_train = load('/content/gdrive/MyDrive/DL_ASG_1/kmnist-train-labels.npz')
x_test  = load('/content/gdrive/MyDrive/DL_ASG_1/kmnist-test-imgs.npz')
y_test  = load('/content/gdrive/MyDrive/DL_ASG_1/kmnist-test-labels.npz')

x_train = torch.tensor(x_train)
y_train = torch.tensor(y_train)
x_test = torch.tensor(x_test)
y_test = torch.tensor(y_test)

# Check if the data is loaded properly
print(f"x_train shape: {x_train.shape}")
print(f"y_train shape: {y_train.shape}")
print(f"x_test shape: {x_test.shape}")
print(f"y_test shape: {y_test.shape}")

[3.5] Using matplotlib display the first image from the train set and its target value

In [ ]:
# Get the first image and its corresponding target value
first_image = x_train[0]
target_value = y_train[0]

# Display the first image
plt.imshow(first_image, cmap='gray')  # Assuming the image is grayscale
plt.title(f"Target Value: {target_value}")
plt.axis('off')  # Hide axes for better visualization
plt.show()

# plt.imshow(x_train[0][0][0],cmap="gray")


# 4. Prepare Data

[4.1] Reshaping the images from the training and testing set to have the channel dimension last. The dimensions should be: (row_number, height, width, channel)

In [ ]:
x_train = x_train.reshape(x_train.shape[0], img_height, img_width, 1)
x_test = x_test.reshape(x_test.shape[0], img_height, img_width, 1)

print(f"x_train reshaped shape: {x_train.shape}")
print(f"x_test reshaped shape: {x_test.shape}")

This reshaping ensures that the data format is consistent with many image processing pipelines where the channel is the last dimension.

[4.2] Cast `x_train` and `x_test` into `float32` decimals

In [ ]:
print(f"x_train data type: {x_train.dtype}")
print(f"x_test data type: {x_test.dtype}")

In [ ]:
x_train = torch.tensor(x_train, dtype=torch.float32)
x_test = torch.tensor(x_test, dtype=torch.float32)
y_train = torch.tensor(y_train, dtype=torch.long)
y_test = torch.tensor(y_test, dtype=torch.long)

print(f"x_train data type: {x_train.dtype}")
print(f"x_test data type: {x_test.dtype}")
print(f"y_train data type: {y_train.dtype}")
print(f"y_test data type: {y_test.dtype}")

Why float32?

It's the standard data type for most deep learning computations in PyTorch.

It helps in reducing memory usage compared to higher precision types like float64, and is generally supported on most hardware accelerators.

[4.3] Standardise the images of the training and testing sets. Originally each image contains pixels with value ranging from 0 to 255. after standardisation, the new value range should be from 0 to 1.

In [ ]:
x_train = x_train / 255.0
x_test = x_test / 255.0

print(f"x_train data type: {x_train.dtype}")
print(f"x_test data type: {x_test.dtype}")

Explanation:
images are likely in a float type with values 0–255, so ToTensor() doesn’t perform the scaling.

How to fix:
Either convert images to uint8 (so ToTensor() will scale them) or manually scale by dividing by 255 in your transform.

In [ ]:
from torchvision.transforms import Compose, ToTensor, Lambda

transform = Compose([
    Lambda(lambda image: image.view(784).float()) # Flatten to a 784-dimensional vector

])

In [ ]:
from torch.utils.data import Dataset

class KMNISTDataset(Dataset):
    def __init__(self, images, labels, transform=None):
        self.images = images
        self.labels = labels
        self.transform = transform

    def __len__(self):
        return len(self.images)

    def __getitem__(self, idx):
        image = self.images[idx]
        label = self.labels[idx]
        # Apply transformation if specified
        if self.transform:
            image = self.transform(image)
        return image, label

# Create instances for training and testing datasets
train_dataset = KMNISTDataset(x_train, y_train, transform=transform)
test_dataset = KMNISTDataset(x_test, y_test, transform=transform)

In [ ]:
# Check the range of pixel values for x_train and x_test
print("x_train min:", x_train.min(), "max:", x_train.max())
print("x_test min:", x_test.min(), "max:", x_test.max())

Range Check:

x_train.min() and x_train.max() will output the minimum and maximum pixel values in your training set.

After standardisation, you should see values that range from 0.0 to around 1.0.

In [ ]:
sample_image, sample_label = train_dataset[0]
print("Sample image shape:", sample_image.shape)

In [ ]:
# Check number of samples in each dataset
print("Number of training samples:", len(train_dataset))
print("Number of testing samples:", len(test_dataset))

# Retrieve a sample from the training dataset
sample_image, sample_label = train_dataset[0]
print("Shape of a sample image:", sample_image.shape)  # Expected: (784,)
print("Sample label:", sample_label)

[4.4] Create a variable called `num_classes` that will take the value 10 which corresponds to the number of classes for the target variable

In [ ]:
num_classes = 10

[4.5] Convert the target variable for the training and testing sets to a binary class matrix of dimension (rows, num_classes).

For example:
- class 0 will become [1, 0, 0, 0, 0, 0, 0, 0, 0, 0]
- class 1 will become [0, 1, 0, 0, 0, 0, 0, 0, 0, 0]
- class 5 will become [0, 0, 0, 0, 0, 1, 0, 0, 0, 0]
- class 9 will become [0, 0, 0, 0, 0, 0, 0, 0, 0, 1]

In [ ]:
example_classes = [0, 1, 5, 9]

for cls in example_classes:
    one_hot_vector = F.one_hot(torch.tensor(cls, dtype=torch.long), num_classes=num_classes).tolist()
    print(f'class {cls}: {one_hot_vector}')

In [ ]:
y_train = F.one_hot(torch.tensor(y_train, dtype=torch.long), num_classes=num_classes)
y_test = F.one_hot(torch.tensor(y_test, dtype=torch.long), num_classes=num_classes)

F.one_hot(..., num_classes=num_classes):
Once the tensor is of the correct type, F.one_hot can convert the class indices to one-hot encoded vectors where each class is represented by a binary vector of length num_classes.

# 5. Define Neural Networks Architecure

[5.1] Set the seed in PyTorch for reproducing results



In [ ]:
torch.manual_seed(42)
np.random.seed(42)

In [ ]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

[5.2] Define the architecture of your Neural Networks and save it into a variable called `model`

In [ ]:
class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork, self).__init__()
        # First fully connected layer: input 784 -> hidden 512
        self.fc1 = nn.Linear(784, 512)
        # Dropout layer to reduce overfitting
        self.dropout1 = nn.Dropout(0.2)
        # Second fully connected layer: hidden 512 -> hidden 256
        self.fc2 = nn.Linear(512, 256)
        # Another dropout layer
        self.dropout2 = nn.Dropout(0.2)
        # Output layer: hidden 256 -> output 10 (one for each class)
        self.fc3 = nn.Linear(256, num_classes)

    def forward(self, x):
        # Pass input through first layer and apply ReLU activation
        x = F.relu(self.fc1(x))
        # Apply dropout
        x = self.dropout1(x)
        # Pass through second layer and apply ReLU
        x = F.relu(self.fc2(x))
        # Apply dropout
        x = self.dropout2(x)
        # Final output layer (logits, no softmax here since loss functions like CrossEntropyLoss expect raw logits)
        x = self.fc3(x)
        return x

# Instantiate the model and save it in the variable 'model'
model = NeuralNetwork()

In [ ]:
model.to(device)
print (model)

# 6. Train Neural Networks

[6.1] Create 2 variables called `batch_size` and `epochs` that will  respectively take the values 128 and 500

In [ ]:
batch_size = 128
epochs = 50 #choose 50 for short run-time

[6.2] Compile the model with the appropriate loss function, the optimiser of your choice and the accuracy metric

In [ ]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001) #optimizer (using Adam with a learning rate of 0.001)


# Define an accuracy metric function.
# Since our targets are one-hot encoded, we first convert them to integer labels using argmax.
def accuracy(outputs, targets):
    # Convert model outputs (logits) to predicted class indices
    predicted = torch.argmax(outputs, dim=1)
    # Convert one-hot encoded targets back to class indices
    actual = torch.argmax(targets, dim=1)
    return (predicted == actual).float().mean()

[6.3] Training model using the number of epochs defined. Calculate the total loss and save it to a variable called total_loss.

In [ ]:
dataloader_train = DataLoader(train_dataset, batch_size = batch_size, shuffle = True)
dataloader_test = DataLoader(test_dataset, batch_size = batch_size, shuffle=True)

In [ ]:
loss_history = []

for i in range(epochs):
    total_loss = 0
    for data, target in dataloader_train:
        data = data.to(device)  # Move data to the same device as the model
        target = target.to(device)  # Move target to the same device as the model
        optimizer.zero_grad() #Zero gradients
        outputs = model(data) # Forward Propagation to get predicted outcome
        loss = criterion(outputs, target.long()) # Compute the loss
        loss.backward()  # Back propagation
        optimizer.step()  # Update the weights
        total_loss += loss.item()

    total_loss /= len(dataloader_train)
    loss_history.append(total_loss)
    print(f"EPOCH {i}: 'Loss' {total_loss:.4f}")


[6.4] Testing model.  Initiate the model.eval() along with torch.no_grad() to turn off the gradients.


In [ ]:
from sklearn.metrics import confusion_matrix
model.eval()
correct = 0
total = 0
input_size = img_height * img_width
# Get the predictions for the test dataset
predicted_labels = []
true_labels = []

with torch.no_grad():
    for data, target in dataloader_test:
        data = data.view(-1, input_size).to(device)
        if target.ndim > 1 and target.size(1) == num_classes:
          target = torch.argmax(target, dim=1)
        target = target.to(device)
        outputs = model(data)
        _, predicted = torch.max(outputs.data, 1)
        total += target.size(0)
        correct += (predicted == target).sum().item()
        predicted_labels.extend(predicted.cpu().tolist())
        true_labels.extend(target.cpu().tolist())

# 7. Analyse Results

[7.1] The performance of your model on the training and testing sets

In [ ]:
print (outputs[0])

In [ ]:
print (outputs[1])

In [ ]:
target[0]

In [ ]:
target[1]

In [ ]:
accuracy = correct / total
print(f'Test Accuracy: {accuracy * 100:.2f}%')

Accuracy Calculation:
The code counts the total number of correct predictions and computes the overall accuracy percentage.

[7.2] Plot the learning curve of your model

In [ ]:
plt.figure(figsize=(10, 6))
plt.plot(range(1, epochs + 1), loss_history, marker='o', label='Training Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.title('Learning Curve')
plt.legend()
plt.grid(True)
plt.show()

[7.3] Confusion matrix on the testing set predictions

In [ ]:
cm = confusion_matrix(true_labels, predicted_labels)

import seaborn as sns

plt.figure(figsize=(10, 8))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', cbar=False)
plt.xlabel('Predicted labels')
plt.ylabel('True labels')
plt.title('Confusion Matrix')
plt.show()


Confusion Matrix:
The lists predicted_labels and true_labels are collected during testing and used to compute the confusion matrix via confusion_matrix from sklearn.metrics.

In [ ]:
from sklearn.metrics import classification_report

report = classification_report(true_labels, predicted_labels, digits=4)
print("\nClassification Report:\n", report)

# Exp 2

In [ ]:
from torch.utils.data import random_split

# Define the ratio for validation (e.g., 20% of the original training data)
val_ratio = 0.2
total_samples = len(train_dataset)
val_size = int(total_samples * val_ratio)
train_size = total_samples - val_size
test_size = len(test_dataset)

# Split the original training dataset into new training and validation datasets
train_dataset, val_dataset = random_split(train_dataset, [train_size, val_size])

# Create DataLoaders for the new training and validation sets
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

print(f"New training set size: {train_size}")
print(f"Validation set size: {val_size}")
print(f"Testing set size: {test_size}")

Explanation

Splitting the Dataset:

We set aside 20% of the original training dataset as a validation set using random_split.

The lengths for training and validation sets are computed based on the total number of samples.

DataLoaders:

We create DataLoaders for both the training and validation datasets.

The training DataLoader uses shuffle=True to mix the data at each epoch, while the validation DataLoader uses shuffle=False since shuffling is not required during evaluation.

*   Sets up an optimizer with weight decay (L2 regularization).
*   Trains the model using the new training DataLoader.
*   Evaluates the model on the validation set at the end of each epoch, calculating both the loss and accuracy.





In [ ]:


# Set up the optimizer with weight decay for L2 regularization.
optimizer = optim.Adam(model.parameters(), lr=0.001, weight_decay=1e-5)

# Define our loss function (using CrossEntropyLoss, so targets must be indices)
criterion = nn.CrossEntropyLoss()

# Training loop with validation evaluation at each epoch.
num_epochs = epochs  # using the same 'epochs' variable as before
train_loss_history = []
val_loss_history = []

for epoch in range(num_epochs):
    model.train()
    train_loss = 0.0
    for data, target in train_loader:
        data = data.to(device)
        # If targets are one-hot encoded, convert them to class indices:
        if target.ndim > 1 and target.size(1) == num_classes:
            target = torch.argmax(target, dim=1)
        target = target.to(device)

        optimizer.zero_grad()
        outputs = model(data)
        loss = criterion(outputs, target)
        loss.backward()
        optimizer.step()
        train_loss += loss.item() * data.size(0)

    avg_train_loss = train_loss / len(train_loader.dataset)
    train_loss_history.append(avg_train_loss)

    model.eval()
    val_loss = 0.0
    correct = 0
    total = 0
    with torch.no_grad():
        for data, target in val_loader:
            data = data.to(device)
            # Convert one-hot targets to indices if needed
            if target.ndim > 1 and target.size(1) == num_classes:
                target = torch.argmax(target, dim=1)
            target = target.to(device)

            outputs = model(data)
            loss = criterion(outputs, target)
            val_loss += loss.item() * data.size(0)

            # Get the predictions
            _, predicted = torch.max(outputs, 1)
            total += target.size(0)
            correct += (predicted == target).sum().item()

    avg_val_loss = val_loss / len(val_loader.dataset)
    val_loss_history.append(avg_val_loss)
    val_accuracy = correct / total * 100

    print(f"Epoch {epoch+1}/{num_epochs} | Train Loss: {avg_train_loss:.4f} | Val Loss: {avg_val_loss:.4f} | Val Accuracy: {val_accuracy:.2f}%")

In [ ]:
plt.figure(figsize=(10, 6))
plt.plot(range(1, epochs + 1), train_loss_history, marker='o', label='Training Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.title('Learning Curve')
plt.legend()
plt.grid(True)
plt.show()

In [ ]:
plt.figure(figsize=(10, 6))
plt.plot(range(1, epochs + 1), val_loss_history, marker='o', label='val Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.title('Learning Curve')
plt.legend()
plt.grid(True)
plt.show()

In [ ]:
from sklearn.metrics import confusion_matrix
model.eval()
correct_2 = 0
total_2 = 0
input_size = img_height * img_width
# Get the predictions for the test dataset
predicted_labels_2 = []
true_labels_2 = []

with torch.no_grad():
    for data, target in val_loader:
        data = data.view(-1, input_size).to(device)
        if target.ndim > 1 and target.size(1) == num_classes:
          target = torch.argmax(target, dim=1)
        target = target.to(device)
        outputs = model(data)
        _, predicted = torch.max(outputs.data, 1)
        total_2 += target.size(0)
        correct_2 += (predicted == target).sum().item()
        predicted_labels_2.extend(predicted.cpu().tolist())
        true_labels_2.extend(target.cpu().tolist())

In [ ]:
accuracy_2 = correct_2 / total_2
print(f'Val Accuracy: {accuracy_2 * 100:.2f}%')

In [ ]:
cm_2 = confusion_matrix(true_labels_2, predicted_labels_2)

plt.figure(figsize=(10, 8))
sns.heatmap(cm_2, annot=True, fmt='d', cmap='Blues', cbar=False)
plt.xlabel('Predicted labels')
plt.ylabel('True labels')
plt.title('Confusion Matrix')
plt.show()

In [ ]:
from sklearn.metrics import classification_report

report_2 = classification_report(true_labels_2, predicted_labels_2, digits=4)
print("\nClassification Report:\n", report_2)

In [ ]:
predicted_labels_3 = []
true_labels_3 = []
correct_3 = 0
total_3 = 0

with torch.no_grad():
    for data, target in test_loader:
        data = data.view(-1, input_size).to(device)
        if target.ndim > 1 and target.size(1) == num_classes:
            target = torch.argmax(target, dim=1)
        target = target.to(device)
        outputs = model(data)
        _, predicted = torch.max(outputs, 1)
        total_3 += target.size(0)
        correct_3 += (predicted == target).sum().item()
        predicted_labels_3.extend(predicted.cpu().tolist())
        true_labels_3.extend(target.cpu().tolist())

In [ ]:
accuracy_3 = correct_3 / total_3
print(f'Test Accuracy: {accuracy_3 * 100:.2f}%')

In [ ]:
cm_3 = confusion_matrix(true_labels_3, predicted_labels_3)

plt.figure(figsize=(10, 8))
sns.heatmap(cm_3, annot=True, fmt='d', cmap='Blues', cbar=False)
plt.xlabel('Predicted labels')
plt.ylabel('True labels')
plt.title('Confusion Matrix')
plt.show()

In [ ]:
report_3 = classification_report(true_labels_3, predicted_labels_3, digits=4)

print("Test Classification Report:\n", report_3)

# Exp 3

In [ ]:
class NeuralNetworkExp3(nn.Module):
    def __init__(self):
        super(NeuralNetworkExp3, self).__init__()
        # Increase neurons: from 784 input, now 1024 neurons in the first hidden layer.
        self.fc1 = nn.Linear(784, 1024)
        self.dropout1 = nn.Dropout(0.5)
        # Second hidden layer: reduce to 512 neurons
        self.fc2 = nn.Linear(1024, 512)
        self.dropout2 = nn.Dropout(0.5)
        # Output layer remains at 10 neurons for the 10 classes.
        self.fc3 = nn.Linear(512, 10)

    def forward(self, x):
        # First layer with ReLU activation and 0.5 dropout
        x = F.relu(self.fc1(x))
        x = self.dropout1(x)
        # Second layer with ReLU activation and 0.5 dropout
        x = F.relu(self.fc2(x))
        x = self.dropout2(x)
        # Final output layer (logits)
        x = self.fc3(x)
        return x

# Instantiate the modified model for Experiment 3.
model_exp3 = NeuralNetworkExp3()

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model_exp3.to(device)

In [ ]:
# Use CrossEntropyLoss (expects targets as class indices)
criterion = nn.CrossEntropyLoss()
# Use Adam optimizer with a weight decay for regularization
optimizer = optim.Adam(model_exp3.parameters(), lr=0.001, weight_decay=1e-5)

# Define number of epochs for training
num_epochs = 50

# Lists to store training and validation loss history for Experiment 3
train_loss_history_exp3 = []
val_loss_history_exp3 = []

for epoch in range(num_epochs):
    # Training phase
    model_exp3.train()  # Set the model to training mode
    running_train_loss = 0.0

    for data, target in train_loader:
        # Move the inputs and targets to the same device as the model
        data = data.to(device)
        # If target is one-hot encoded, convert it to class indices
        if target.ndim > 1 and target.size(1) == num_classes:
            target = torch.argmax(target, dim=1)
        target = target.to(device)

        optimizer.zero_grad()         # Zero out gradients from previous iteration
        outputs = model_exp3(data)      # Forward pass
        loss = criterion(outputs, target)  # Compute loss
        loss.backward()               # Backward pass
        optimizer.step()              # Update weights

        running_train_loss += loss.item() * data.size(0)  # Accumulate batch loss

    epoch_train_loss = running_train_loss / len(train_loader.dataset)
    train_loss_history_exp3.append(epoch_train_loss)

    # Validation phase
    model_exp3.eval()  # Set the model to evaluation mode
    running_val_loss = 0.0
    with torch.no_grad():
        for data, target in val_loader:
            data = data.to(device)
            if target.ndim > 1 and target.size(1) == num_classes:
                target = torch.argmax(target, dim=1)
            target = target.to(device)

            outputs = model_exp3(data)
            loss = criterion(outputs, target)
            running_val_loss += loss.item() * data.size(0)

    epoch_val_loss = running_val_loss / len(val_loader.dataset)
    val_loss_history_exp3.append(epoch_val_loss)

    print(f"Epoch {epoch+1}/{num_epochs} | Train Loss: {epoch_train_loss:.4f} | Val Loss: {epoch_val_loss:.4f}")

print("Training complete!")


In [ ]:
plt.figure(figsize=(10, 6))
plt.plot(range(1, num_epochs + 1), train_loss_history_exp3, marker='o', label='Training Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.title('Learning Curve for Experiment 3')
plt.legend()
plt.grid(True)
plt.show()

In [ ]:
plt.figure(figsize=(10, 6))
plt.plot(range(1, num_epochs + 1), val_loss_history_exp3, marker='o', label='Val Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.title('Learning Curve for Experiment 3')
plt.legend()
plt.grid(True)
plt.show()

In [ ]:
from sklearn.metrics import confusion_matrix, classification_report

model_exp3.eval()
input_size = img_height * img_width
predicted_labels_val = []
true_labels_val = []
correct_val = 0
total_val = 0

with torch.no_grad():
    for data, target in val_loader:
        data = data.view(-1, input_size).to(device)
        if target.ndim > 1 and target.size(1) == num_classes:
            target = torch.argmax(target, dim=1)
        target = target.to(device)
        outputs = model_exp3(data)
        _, predicted = torch.max(outputs, 1)
        total_val += target.size(0)
        correct_val += (predicted == target).sum().item()
        predicted_labels_val.extend(predicted.cpu().tolist())
        true_labels_val.extend(target.cpu().tolist())

In [ ]:
accuracy_val = correct_val / total_val
print(f'Val Accuracy: {accuracy_val * 100:.2f}%')

In [ ]:
cm_val = confusion_matrix(true_labels_val, predicted_labels_val)

plt.figure(figsize=(10, 8))
sns.heatmap(cm_val, annot=True, fmt='d', cmap='Blues', cbar=False)
plt.xlabel('Predicted labels')
plt.ylabel('True labels')
plt.title('Confusion Matrix')
plt.show()

In [ ]:
report_val = classification_report(true_labels_val, predicted_labels_val, digits=4)
print("\nClassification Report:\n", report_val)

In [ ]:
predicted_labels_test = []
true_labels_test = []
correct_test = 0
total_test = 0

with torch.no_grad():
    for data, target in test_loader:
        data = data.view(-1, input_size).to(device)
        if target.ndim > 1 and target.size(1) == num_classes:
            target = torch.argmax(target, dim=1)
        target = target.to(device)
        outputs = model_exp3(data)
        _, predicted = torch.max(outputs, 1)
        total_test += target.size(0)
        correct_test += (predicted == target).sum().item()
        predicted_labels_test.extend(predicted.cpu().tolist())
        true_labels_test.extend(target.cpu().tolist())


In [ ]:
accuracy_test = correct_test / total_test
print(f'Test Accuracy: {accuracy_test * 100:.2f}%')

In [ ]:
cm_test = confusion_matrix(true_labels_test, predicted_labels_test)

plt.figure(figsize=(10, 8))
sns.heatmap(cm_test, annot=True, fmt='d', cmap='Blues', cbar=False)
plt.xlabel('Predicted labels')
plt.ylabel('True labels')
plt.title('Confusion Matrix')
plt.show()

In [ ]:
report_test = classification_report(true_labels_test, predicted_labels_test, digits=4)

print("Test Classification Report:\n", report_test)

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.patches as patches

# Lists to store images, true labels, and predictions for correct and incorrect cases
correct_images = []
correct_preds = []
correct_labels = []

incorrect_images = []
incorrect_preds = []
incorrect_labels = []

model_exp3.eval()
input_size = img_height * img_width

with torch.no_grad():
    for data, target in test_loader:
        # Save original images for visualization
        original_images = data.clone()

        # Prepare data for model (flatten)
        data_flat = data.view(-1, input_size).to(device)
        if target.ndim > 1 and target.size(1) == num_classes:
            target = torch.argmax(target, dim=1)
        target = target.to(device)

        # Get predictions
        outputs = model_exp3(data_flat)
        _, predicted = torch.max(outputs, 1)

        # Loop through the batch and separate correct and incorrect predictions
        for i in range(data.shape[0]):
            # Get the image from the original batch and prepare it for plotting
            img = original_images[i].cpu()
            # If the image is flattened, reshape it to (28,28)
            if img.ndim == 1 and img.numel() == 784:
                img = img.view(img_height, img_width)
            else:
                img = img.squeeze()

            true_label = target[i].cpu().item()
            pred_label = predicted[i].cpu().item()

            if true_label == pred_label:
                if len(correct_images) < 10:  # Store up to 10 examples
                    correct_images.append(img)
                    correct_preds.append(pred_label)
                    correct_labels.append(true_label)
            else:
                if len(incorrect_images) < 10:  # Store up to 10 examples
                    incorrect_images.append(img)
                    incorrect_preds.append(pred_label)
                    incorrect_labels.append(true_label)

            # Stop if we have enough examples from both categories
            if len(correct_images) >= 10 and len(incorrect_images) >= 10:
                break
        if len(correct_images) >= 10 and len(incorrect_images) >= 10:
            break

# Visualize Correct Predictions with green border
plt.figure(figsize=(15, 4))
for i, img in enumerate(correct_images):
    ax = plt.subplot(2, 10, i+1)
    plt.imshow(img, cmap='gray')
    plt.title(f"True: {correct_labels[i]}\nPred: {correct_preds[i]}")
    plt.axis('off')
    # Add a green border rectangle
    rect = patches.Rectangle((0, 0), img.shape[1], img.shape[0], fill=False, edgecolor='green', linewidth=3)
    ax.add_patch(rect)
plt.suptitle("Correct Predictions", fontsize=16)
plt.show()

# Visualize Incorrect Predictions with red border
plt.figure(figsize=(15, 4))
for i, img in enumerate(incorrect_images):
    ax = plt.subplot(2, 10, i+1)
    plt.imshow(img, cmap='gray')
    plt.title(f"True: {incorrect_labels[i]}\nPred: {incorrect_preds[i]}")
    plt.axis('off')
    # Add a red border rectangle
    rect = patches.Rectangle((0, 0), img.shape[1], img.shape[0], fill=False, edgecolor='red', linewidth=3)
    ax.add_patch(rect)
plt.suptitle("Incorrect Predictions", fontsize=16)
plt.show()
